In [ ]:
# PACKAGE INSTALLATION

%pip install jupyter stanza supabase dotenv

In [24]:
# LOCAL IMPORTS

import importlib
import romanizer
import db

importlib.reload(romanizer)     # Reload romanizer from korean
importlib.reload(db)

<module 'db' from '/home/boddah/Documents/Stanza/db.py'>

In [5]:
# DATABASE

import os
from dotenv import load_dotenv
from supabase import create_client, Client

load_dotenv()

url: str = os.environ.get("SUPABASE_URL")
key: str = os.environ.get("SUPABASE_KEY")

supabase: Client = create_client(url, key)

In [ ]:
# MODEL DOWNLOAD

import stanza

language = 'ko'
stanza.download(language)       # Download model
nlp = stanza.Pipeline(language) # Initialize neural pipeline

In [ ]:
# PHRASE TOKENIZATION
phrase = "무엇을 먹을 거예요?"

doc = nlp(phrase)
dict = doc.to_dict()

print(doc.sentences)

[[
  {
    "id": 1,
    "text": "무엇을",
    "lemma": "무엇+을",
    "upos": "PRON",
    "xpos": "npd+jco",
    "head": 2,
    "deprel": "obj",
    "start_char": 0,
    "end_char": 3
  },
  {
    "id": 2,
    "text": "먹을",
    "lemma": "먹+ㄹ",
    "upos": "VERB",
    "xpos": "pvg+etm",
    "head": 3,
    "deprel": "ccomp",
    "start_char": 4,
    "end_char": 6
  },
  {
    "id": 3,
    "text": "거예요",
    "lemma": "거+이+에요",
    "upos": "VERB",
    "xpos": "nbn+jp+ef",
    "head": 0,
    "deprel": "root",
    "start_char": 7,
    "end_char": 10,
    "misc": "SpaceAfter=No"
  },
  {
    "id": 4,
    "text": "?",
    "lemma": "?",
    "upos": "PUNCT",
    "xpos": "sf",
    "head": 3,
    "deprel": "punct",
    "start_char": 10,
    "end_char": 11,
    "misc": "SpaceAfter=No"
  }
]]


In [76]:
from color import blue, bold, cyan, green, purple, red


for sentences in dict:
    print("Phrase: ", phrase)
    for words in sentences:
        print(bold(cyan("Text: ")), words['text'], '|', blue(romanizer.hangul(words['text'])))

        upos = db.getUpos(supabase, words['upos'])
        print(green("Part of Speech: "), upos)

        print(red("Morphemes: "))
        morphemes = words['lemma'].split('+')

        tags = words['xpos'].split('+')
        xpos = db.getXpos(supabase, tags)

        morphemes_info = db.getMorphemes(supabase, morphemes, tags)

        for morpheme, tag, xpos_label, info in zip(morphemes, tags, xpos, morphemes_info):
            print(f"    '{morpheme}': '{tag} => {xpos_label.title()}' | {bold(red('Translation'))}: {info['translation']}")

        translation = db.getTranslation(supabase, words['text'], words['upos'], morphemes_info)
        print(purple("Translation: "), translation)
        print("─"*30, '\n')


Phrase:  무엇을 먹을 거예요?
Text:  무엇을 | mueoseul
Part of Speech:  Pronoun
Morphemes: 
    '무엇': 'npd => Demonstrative Noun/Pronoun' | Translation: what
    '을': 'jco => Object Case Particle' | Translation: (object particle)
Translation:  what (object)
────────────────────────────── 

Text:  먹을 | meokeul
Part of Speech:  Verb
Morphemes: 
    '먹': 'pvg => General Verb Root' | Translation: to eat
    'ㄹ': 'etm => Adnominal Suffix' | Translation: -going to (future/intentional modifier)
Translation:  going to eat
────────────────────────────── 

Text:  거예요 | geoyeyo
Part of Speech:  Verb
Morphemes: 
    '거': 'nbn => Nominal Bound Noun' | Translation: thing
    '이': 'jp => Subject Particle' | Translation: (subject particle)
    '에요': 'ef => Final Ending' | Translation: to be (informal high)
Translation:  it is the thing
────────────────────────────── 

Text:  ? | ?
Part of Speech:  Punctuation
Morphemes: 
    '?': 'sf => Final Punctuation Mark' | Translation: ?
Translation:  ?
────────────────────